# Part One

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob

In [13]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds41_42_config.yml'

In [14]:
config_filename

'./ds41_42_config.yml'

In [15]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [36]:
first_ID = config['inputs']['first_ID']

In [37]:
second_ID = config['inputs']['second_ID']

In [38]:
first_ID_datafiles = config[first_ID].values()

In [39]:
second_ID_datafiles = config[second_ID].values()

In [40]:
# If collecting user input from Jupyter Notebook
# choice1 = input("Please input your first choice: ")
# choice1

# Downloading SQL Data

In [55]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [56]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [57]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [58]:
# SQl Data at different levels 
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction']
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa']

## Downloading the first_ID Data

In [68]:
# first_ID Data Frame
first_ID_df = pd.DataFrame()
for file_name in first_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    first_ID_df = first_ID_df.append(working_df)

In [69]:
first_ID_df

,mgra,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,...,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,year
0,1,3331,19,19,0,0,18,18,0,0,...,0,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016
1,2,3331,35,35,0,0,34,34,0,0,...,0,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016
2,3,3358,52,52,0,0,52,52,0,0,...,0,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016
3,4,3358,30,30,0,0,30,30,0,0,...,0,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016
4,5,3358,28,28,0,0,28,28,0,0,...,0,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22997,22998,1290,92,92,0,0,87,87,0,0,...,0,0,14,1,1,2.6193,41.241522,26.863578,41.241522,2050
22998,22999,1290,0,0,0,0,0,0,0,0,...,0,0,14,1,1,2.3703,35.842780,30.635095,35.842780,2050
22999,23000,1290,131,131,0,0,126,126,0,0,...,0,0,14,1,1,2.1721,28.735275,16.109816,28.735275,2050
23000,23001,1254,85,85,0,0,82,82,0,0,...,0,0,14,1,1,2.7063,41.006144,28.126056,41.006144,2050


In [70]:
#Making it original 
first_ID_df.columns = [column_name + f'_{first_ID}' for column_name in first_ID_df.columns]

In [71]:
import copy 
comparison_first_ID_no_geozone = copy.deepcopy(first_ID_df)

In [72]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data = comparison_first_ID_no_geozone.merge(jur_level, how = 'left', left_on = f'mgra_{first_ID}', right_on = 'mgra')

In [73]:
# Need to drop one of the mgra columns
comparison_first_ID_processed_data

,mgra_DS35,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,...,truckregiontype_DS35,district27_DS35,milestocoast_DS35,acres_DS35,effective_acres_DS35,land_acres_DS35,year_DS35,mgra,geotype,geozone
0,1,3331,19,19,0,0,18,18,0,0,...,1,27,3.7997,16.615444,12.961482,16.615444,2016,1,jurisdiction,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,1,27,3.7997,16.615444,12.961482,16.615444,2016,1,jurisdiction,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,1,27,3.9761,19.519185,19.519185,19.519185,2016,2,jurisdiction,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,1,27,3.9761,19.519185,19.519185,19.519185,2016,2,jurisdiction,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,1,27,4.1939,27.845124,26.867938,27.845124,2016,3,jurisdiction,San Diego
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355103,22998,1290,92,92,0,0,87,87,0,0,...,1,1,2.6193,41.241522,26.863578,41.241522,2050,22998,jurisdiction,Carlsbad
355104,22999,1290,0,0,0,0,0,0,0,0,...,1,1,2.3703,35.842780,30.635095,35.842780,2050,22999,jurisdiction,Carlsbad
355105,23000,1290,131,131,0,0,126,126,0,0,...,1,1,2.1721,28.735275,16.109816,28.735275,2050,23000,jurisdiction,Carlsbad
355106,23001,1254,85,85,0,0,82,82,0,0,...,1,1,2.7063,41.006144,28.126056,41.006144,2050,23001,jurisdiction,Carlsbad


In [54]:
# first_ID_df.to_csv(f"/comparison_{first_ID}_processed_data")

# Downloading  the second_ID Data

In [76]:
# second_id Data Frame
second_ID_df = pd.DataFrame()
for file_name in second_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    second_ID_df = second_ID_df.append(working_df)

In [77]:
#Making it original 
second_ID_df.columns = [x + f'_{second_ID}' for x in second_ID_df.columns]

In [79]:
comparison_second_ID_no_geozone = copy.deepcopy(second_ID_df)

In [80]:
# Adding SQl Data to first_id_df
comparison_second_ID_processed_data = comparison_second_ID_no_geozone.merge(jur_level, how = 'left', left_on = f'mgra_{second_ID}', right_on = 'mgra')

In [81]:
# Need to drop one of the mgra columns
comparison_second_ID_processed_data

,mgra_DS41,taz_DS41,hs_DS41,hs_sf_DS41,hs_mf_DS41,hs_mh_DS41,hh_DS41,hh_sf_DS41,hh_mf_DS41,hh_mh_DS41,...,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41,mgra,geotype,geozone
0,1,3331,19,19,0,0,18,18,0,0,...,1,27,3.7997,16.615444,12.961482,16.615444,2016,1,jurisdiction,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,1,27,3.7997,16.615444,12.961482,16.615444,2016,1,jurisdiction,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,1,27,3.9761,19.519185,19.519185,19.519185,2016,2,jurisdiction,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,1,27,3.9761,19.519185,19.519185,19.519185,2016,2,jurisdiction,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,1,27,4.1939,27.845124,26.867938,27.845124,2016,3,jurisdiction,San Diego
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355103,22998,1290,92,92,0,0,87,87,0,0,...,1,1,2.6193,41.241522,26.863578,41.241522,2050,22998,jurisdiction,Carlsbad
355104,22999,1290,0,0,0,0,0,0,0,0,...,1,1,2.3703,35.842780,30.635095,35.842780,2050,22999,jurisdiction,Carlsbad
355105,23000,1290,131,131,0,0,126,126,0,0,...,1,1,2.1721,28.735275,16.109816,28.735275,2050,23000,jurisdiction,Carlsbad
355106,23001,1254,85,85,0,0,81,81,0,0,...,1,1,2.7063,41.006144,28.126056,41.006144,2050,23001,jurisdiction,Carlsbad


In [82]:
# second_ID_df.to_csv(f"/comparison_{second_ID}_processed_data")